In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import glob
import os
import csv
from collections import OrderedDict
import math
import datetime 
import warnings

dataset can be varied in any field. here, lets assume, dataset is a haemodynamid data and we have many paitients
where their heart rate are changing over time

In [ ]:
df = pd.read_csv('Time_series_dataset.csv')


In [ ]:
# make the time and date in readable format for pandas analysis
# sort the data based on time for further analysis
df['t'] = pd.to_datetime(df['t'])
df= df.sort_values(by=['ID','t'])

In [ ]:
#changing the time intervals with the average (if needed)
df= df.groupby([pd.Grouper(key='t', freq='1Min'), 'ID']).mean().reset_index()

In [ ]:
# if a column needed to be floated
df['variable'] = df['variable'].astype(float)


In [ ]:
# for CUMEAN we need to subtract the updated mean from each target value
df_updatedMean = df.set_index('t').groupby(['ID'])['variable'].rolling('2D').mean()
df_updatedMean=df_updatedMean.reset_index().rename(columns={'variable':'UpdatedMean'})


In [ ]:
#merging back the new updated mean to orginal file
df_merged = pd.merge(df_updatedMean, df,  on=['ID','t'])

In [ ]:
#subtract the updated mean from target values
df_merged ['target_minus_updatedMean']= df_merged ['variable'] - df_merged ['UpdatedMean]


In [ ]:
# now use cusum for updated target values
df_merged ['CUMEAN']=df_merged.groupby('ID')['target_minus_updatedMean'].cumsum(skipna = True)


In [ ]:
#CUMEAN and CUSUM can be more useful by using upper/lower control limit (based on 3 sigma)


In [ ]:
# FIndind mean and STD for ucl and lcl
mean= df.groupby('ID')['variable'].mean().reset_index().rename(columns={'variable':'mean_variable'})
std= df.groupby('ID')['variable'].std().reset_index().rename(columns={'variable':'std_variable'})

In [ ]:
#lcl ucl
mean['ucl'] = mean['mean_variable'] + 3* std['std_variable']
mean['lcl'] = mean['mean_variable'] - 3* std['std_variable']


In [ ]:
#merging back the new varibles to the dataset
df_merged = pd.merge(df_merged, mean,  on=['ID'])

In [ ]:
# all ready now